![](problem_statements/E051.png)

# Input

In [ ]:
N,K,L = map(int,input().strip().split(' '))

# Steps

First, find all the $N$-digit primes, by Sieve of Eratosthenes...

In [ ]:
def get_primes(num):
    "Return a list of all primes <=num"
    
    #Start with the initialization of all numbers from 1 to num as prime=True
    prime = [True if i>=2 else False for i in range(num)]
    
    #Now, run a index from 2 to sqrt(num)
    i = 2
    while i*i<num:
        #If i is prime, all multiples of i which are <=num will be composite...
        if prime[i]:
            for j in range(i*2,num,i):
                prime[j]=False
        i+=1
    #Now, list all the primes - i.e. all n such that prime[n]==True
    #Here, we ensure that we are looking for only n-digits
    ans = []
    for i in range(num//10,num):
        if prime[i]:
            ans.append(str(i))
    return ans

In [ ]:
slct_primes_list = get_primes(10**N)

In [ ]:
#The mode of the digit distribution needs to be atleast K (apply this for k>=3 to reduce workload)
if K>=3:
    max_freq = lambda x: max([x.count(str(l)) for l in range(10)])
    slct_primes_list = [x for x in slct_primes_list if max_freq(x)>=K]

Now, we need to group these in families, based on how they differ from each other in digits. The problem can be broken down into:
1. Select the $K$ positions along the N-digit number where substitutions will be done
2. For a particular selection above, start from smallest prime number and search for other primes that will be substitutions along that selection.

In [ ]:
from itertools import combinations
#For L>4, the last digit cannot be a substitution position
if L>4:
    substitution_positions_list = list(combinations(list(range(N-1)),K))
else:
    substitution_positions_list = list(combinations(list(range(N)),K))

# Solution

Now run a loop on all substitution positions, and get the possible family (if one exists) for that substitution position

# Optimization

The most time consuming part is the primality checking done for each element in *possible_members*. We can reduce the time and computational effort for this by doing primality_checking using *slct_primes_list* - if the member is in the list or not. As this is a sorted list, we can use binary search. Write a definition to do binary search:

In [ ]:
def binary_search(item,sorted_list):
    "Function to return the position of the item in sorted_list, if it exists in sorted_list"
    start,end = 0,len(sorted_list)-1
    while start<=end:
        midpoint = (start+end)//2
        test = sorted_list[midpoint]
        if item==test:
            return test
        elif item<test:
            end = midpoint - 1
        elif item>test:
            start = midpoint + 1

We can also optimize the search for families by eliminating numbers already considered in *slct_primes_list*, as we search this space. Similar minor optimizations are made

In [ ]:
from collections import Counter

def get_family_optimized(sub_pos):
    "get_family optimized with prime checking using several optimization techniques"
    
    #We need only those with equal digits at substitution positions...
    ls = [p for p in slct_primes_list if all([p[s]==p[sub_pos[0]] for s in sub_pos])]
    
    if K==1:
        get_pattern = lambda x: x[:sub_pos[0]] + '*' + x[sub_pos[0]+1:]
    elif K==2:
        get_pattern = lambda x: x[:sub_pos[0]] +'*' + x[sub_pos[0]+1:sub_pos[1]] + '*' + x[sub_pos[1]+1:]
    elif K==3:
        get_pattern = lambda x: x[:sub_pos[0]] +'*' + x[sub_pos[0]+1:sub_pos[1]] + '*' + \
        x[sub_pos[1]+1:sub_pos[2]] + '*' + x[sub_pos[2]+1:]
    else:
        get_pattern = lambda x: ''.join(['*' if i in sub_pos else x[i] for i in range(N)])
    
    #The corresponding patterns for all elements in ls
    all_patterns = [get_pattern(p) for p in ls]
    pattern_counter = Counter(all_patterns)
    
    #If none of the patterns have >=L primes, go no further...
    if pattern_counter[max(pattern_counter,key = lambda x:pattern_counter[x])]<L:
        return None
    
    #Keep only those with pattern occurring >=L times..
    ls = [x for x,y in zip(ls,all_patterns) if pattern_counter[y]>=L]
    
    mask = {k:True for k in ls}
    for first_element in filter(lambda x:mask[x],ls):
    
        #We dont need to sonsider sequences with first element > current_min_first_element
        if first_element>current_min_first_element:
            break
        
        #Check if all the digits in sub_pos are same...
        digit_at_sub = int(first_element[sub_pos[0]])
        
        #Check if a L-length family is even possible
        if digit_at_sub+L>10:
            mask[first_element] = False
            continue
        
        pattern = get_pattern(first_element)
        possible_members = [pattern.replace('*',str(d)) for d in range(digit_at_sub,10)]

        family = [binary_search(p,ls) for p in possible_members if binary_search(p,ls)]
        mask.update({f:False for f in family})

        if len(family)>=L:
            return family[:L]

    else:
        return None

In [ ]:
families = []
current_min_first_element = slct_primes_list[-1]
for sub in substitution_positions_list:
    ans = get_family_optimized(sub)
    if ans:
        families.append(ans)
        current_min_first_element = min(ans[0],current_min_first_element)
try:
    print(' '.join(min(families)))
except:
    pass